## Reference Materials

https://github.com/marcotcr/lime?tab=readme-ov-file

In [2]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function


In [3]:
import pandas as pd
import argparse, os
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import DataLoader 

c:\Users\celin\miniconda3\envs\videogpt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class Q_vecDataset(torch.utils.data.Dataset):
  def __init__(self, X_rus, y_rus, train):
        self.feature = X_rus
        self.target = y_rus
        self.train = train

  def __len__(self):
        return self.target.shape[0]

  def __getitem__(self, index):
        X = self.feature.iloc[[index]]
        X = X.to_numpy()
        y = self.target.iloc[[index]]
        y= y.to_numpy()

        return X, y

In [5]:
def prepare_data():
    q_cleaned_old = pd.read_csv('data/data_vectorized_240228.csv')
    q_cleaned_old.drop(['ia_status_Facility Study', 'ia_status_Feasibility Study',
        'ia_status_IA Executed', 'ia_status_Operational',
        'ia_status_System Impact Study', 'Unnamed: 0'], axis = 1, inplace=True)

    exempt = []
    for col in list(q_cleaned_old.columns):
        if q_cleaned_old[col].max() < 1:
            exempt.append(col)
    q_cleaned_old.drop(columns = exempt, inplace=True)
    
    # Use batch normalization here - subtract by mean of data + divide by variance
    scaler = StandardScaler()
    scaler.fit(q_cleaned_old)
    q_cleaned_array = scaler.transform(q_cleaned_old)
    q_cleaned = pd.DataFrame(q_cleaned_array, columns=q_cleaned_old.columns)
    

    features = q_cleaned.drop(['ia_status_Withdrawn'], axis = 1)
    target = q_cleaned_old['ia_status_Withdrawn']

    seed = 42

    rus = RandomUnderSampler(random_state=seed)
    X_rus, y_rus= rus.fit_resample(features, target)
    X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus,
                                                            test_size = 0.2,
                                                            random_state = seed)
    return X_train, X_test, y_train, y_test

In [9]:
%cd ..

c:\Users\celin\git\Quennect


In [10]:
X_train, X_test, y_train, y_test = prepare_data()

In [21]:
classes = [0, 1]

In [18]:
q_cleaned_old = pd.read_csv('data/data_vectorized_240228.csv')
q_cleaned_old.drop(['ia_status_Facility Study', 'ia_status_Feasibility Study',
        'ia_status_IA Executed', 'ia_status_Operational',
        'ia_status_System Impact Study', 'Unnamed: 0'], axis = 1, inplace=True)
q_cleaned_old.drop(['ia_status_Withdrawn'], axis = 1)

,q_year,prop_year,region_CAISO,region_ERCOT,region_ISO-NE,region_MISO,region_NYISO,region_PJM,region_SPP,region_Southeast (non-ISO),...,util_tsgt,util_ugi,util_unknown,util_upper peninsula power company,util_vea,util_wabash valley power,util_wisconsin electric power company,util_wisconsin public service corporation,"util_wolverine power supply cooperative, inc.",util_xcel energy
0,2016.0,2018.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2008.0,2011.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2019.0,2023.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2009.0,2010.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020.0,2023.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5475,2005.0,2007.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5476,2008.0,2010.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5477,2003.0,2006.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5478,2007.0,2007.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
feature_names = q_cleaned_old.columns

In [22]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=feature_names, class_names=classes, discretize_continuous=True)

InvalidIndexError: (slice(None, None, None), 0)